In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
import networkx as nx
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize

%matplotlib inline

In [2]:
nltk.download('wordnet')
nltk.download('punkt')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anjaneyatripathi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anjaneyatripathi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def load_text(file_name):
    
    with open(file_name, 'r') as file:
        text = file.read()
        
    sentences = sent_tokenize(text)
    
    return sentences

In [4]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [5]:
def create_bow_dictionary(sentences):
    
    # preprocessing the sentences by stemming them
    processed_docs = []
    for doc in sentences:
        processed_docs.append(preprocess(doc))
        
    # forming the dictionary
    dictionary = gensim.corpora.Dictionary(processed_docs)
    
    # creating the BOWs 
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    
    return dictionary, bow_corpus

In [6]:
# creating the LDA model
def create_model(topics, bow_corpus, dictionary):
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics = topics, id2word = dictionary, passes = 10, workers = 2)
    
    return lda_model

In [7]:
# parsing the result of the topics generated
def extract_keywords(s):
    
    words=[]
    status=False
    word=''
    for ch in s:
        if(ch=='"'):
            status = not status;
        elif(ch=='+'):
            status=False
            words.append(word)
            word=''
        elif(status):
            word+=ch
    words.append(word)
    
    return words

In [8]:
sentences = load_text('text.txt')
topics = 3

In [9]:
dictionary, bow_corpus = create_bow_dictionary(sentences)

In [10]:
lda_model = create_model(3, bow_corpus, dictionary)

In [11]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.053*"trade" + 0.052*"chan" + 0.037*"trial" + 0.037*"insid" + 0.037*"convict" + 0.037*"charg" + 0.037*"wang" + 0.021*"drug" + 0.021*"compani" + 0.021*"accord"


Topic: 1 
Words: 0.056*"wang" + 0.032*"complaint" + 0.032*"secur" + 0.032*"charg" + 0.032*"offic" + 0.032*"exchang" + 0.032*"antifraud" + 0.032*"provis" + 0.032*"thereund" + 0.032*"seek"


Topic: 2 
Words: 0.070*"chan" + 0.049*"tip" + 0.049*"alleg" + 0.049*"akebia" + 0.049*"wang" + 0.028*"drug" + 0.028*"approxim" + 0.028*"base" + 0.028*"later" + 0.028*"success"




In [14]:
def classify_topics(topics, lda_model, sentences):

    keyword_dict = {}

    for i in range(topics):
        keyword_dict[i] = extract_keywords(lda_model.print_topics(-1)[i][1])
        
    TOPICS = {}

    for i in range(topics):
        TOPICS[i] = []
    
    for sent in sentences:
        processed = preprocess(sent)
        occurences = [0] * topics
        for word in processed:
            for i in range(topics):
                if(word in keyword_dict[i]):
                    occurences[i]+=1
        top_hit = max(occurences)
        for i in range(topics):
            if(occurences[i]==top_hit):
                print(i)
                TOPICS[i].append(sent)
        print(sent)
    
    return keyword_dict, TOPICS

In [19]:
keyword_dict, TOPICS = classify_topics(topics, lda_model, sentences)

0
According to the SEC's complaint, Wang tipped his close friend, Jason "Schultz" Chan, to trade Merrimack securities in advance of the company announcing positive drug trial results.
2
The SEC further alleges that Chan later returned the favor and tipped Wang with nonpublic information about a successful drug trial conducted by Chan's employer, Akebia Therapeutics, Inc.
2
The complaint alleges Wang made approximately $108,000 trading Akebia securities based on Chan's tips.
0
The SEC previously charged Chan with insider trading in connection with this investigation.
0
Both Wang and Chan were charged with insider trading by the U.S. Attorney's Office for the District of Massachusetts and were criminally convicted after a jury trial.
0
They are currently in the process of appealing their convictions.
1
The SEC's complaint charges Wang with violating the antifraud provisions of Section 10(b) of the Securities and Exchange Act of 1934 and Rule 10b-5 thereunder and seeks disgorgement of Wan

In [21]:
for i in range(topics):
    print('Topic #',i)
    print(TOPICS[i])

Topic # 0
['According to the SEC\'s complaint, Wang tipped his close friend, Jason "Schultz" Chan, to trade Merrimack securities in advance of the company announcing positive drug trial results.', 'The SEC previously charged Chan with insider trading in connection with this investigation.', "Both Wang and Chan were charged with insider trading by the U.S. Attorney's Office for the District of Massachusetts and were criminally convicted after a jury trial.", 'They are currently in the process of appealing their convictions.']
Topic # 1
["The SEC's complaint charges Wang with violating the antifraud provisions of Section 10(b) of the Securities and Exchange Act of 1934 and Rule 10b-5 thereunder and seeks disgorgement of Wang's illegal profits, a civil penalty, an injunction, and an officer-and-director bar."]
Topic # 2
["The SEC further alleges that Chan later returned the favor and tipped Wang with nonpublic information about a successful drug trial conducted by Chan's employer, Akebia 